# Building a Database
Using the insights from the last few notebooks, this notebook will create a Database in Postgres with SQL.

## Targets
The targets are the same ones downloaded in the last notebook, which were saved as 'BechdelData.csv'. That file can be loaded and stored in the Database more or less directly:

In [192]:
import json
import ast
import psycopg2
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings("ignore")
from urllib.request import urlopen
from urllib.error import HTTPError
from concurrent.futures import ThreadPoolExecutor

In [2]:
targets = pd.read_csv('BechdelData.csv')
targets = targets[['id', 'imdbid', 'title', 'year', 'rating']]
targets['imdbid'].fillna(-99, inplace=True)
targets['imdbid'] = targets['imdbid'].astype(int)
targets.to_csv('targets.csv', header=False, index=False, sep='~')

In [46]:
conn = psycopg2.connect(dbname='bechdel_test', user='postgres', password='guest')
cur = conn.cursor()

In [4]:
cur.execute(
    'CREATE TABLE bechdel_ratings('
    'bechdel_id      INT, '
    'imdb_id         INT, '
    'title           VARCHAR(255), '
    'year            SMALLINT, '
    'rating          SMALLINT);',
)

In [5]:
cur.copy_from(f:=open('targets.csv', 'r'), 'bechdel_ratings', columns=['bechdel_id', 'imdb_id', 'title', 'year', 'rating'], sep='~')

In [11]:
conn.commit()
cur.execute('SELECT * FROM bechdel_ratings')
cur.fetchall()

[(9602, 3155794, 'Passage de Venus', 1874, 0),
 (9804, 14495706, 'La Rosace Magique', 1877, 0),
 (9603, 2221420, 'Sallie Gardner at a Gallop', 1878, 0),
 (9806, 12592084, 'Le singe musicien', 1878, 0),
 (9816, 7816420, 'Athlete Swinging a Pick', 1881, 0),
 (9831, 5459794, 'Buffalo Running', 1883, 0),
 (9832, 8588366, 'L&#39;homme machine', 1885, 0),
 (9614, 2075247, 'Man Walking Around the Corner', 1887, 0),
 (9836, 8133192, 'Cockatoo Flying', 1887, 0),
 (9837, 7411790, 'Child Carrying Flowers to Woman', 1887, 0),
 (9838, 7541160, 'Jumping Over a Man&#39;s Back-Leapfrog', 1887, 0),
 (9841, 7754902, 'Man Riding Jumping Horse', 1887, 0),
 (9902, 8361552, 'Baboon Climbing a Pole', 1887, 0),
 (8040, 392728, 'Roundhay Garden Scene', 1888, 0),
 (9615, 1758563, 'Accordion Player', 1888, 0),
 (9616, 343112, 'Traffic Crossing Leeds Bridge', 1888, 0),
 (9675, 3274100, 'Pferd und Reiter Springen uber ein Hindernis', 1888, 0),
 (9947, 2116853, 'Brighton Street Scene', 1888, 0),
 (9619,
  466876,
 

## TMDb Metadata
This .csv created using the code in the previous notebook:

In [467]:
df = pd.read_csv('tmdb_data_targets.csv', index_col=0, converters={"genre_ids": ast.literal_eval})
df['imdbid'] = df['imdbid'].fillna(-99).astype(int)
df1 = df[['id', 'imdbid', 'title', 'original_language', 'release_date', 'popularity', 'vote_average', 'vote_count']]

In [471]:
cur.execute(
    'CREATE TABLE tmdb_data('
    'tmdb_id            INT, '
    'imdb_id            INT, '
    'title              VARCHAR(255),'
    'original_language  VARCHAR(2),'
    'release_date       VARCHAR(10),'
    'popularity         NUMERIC(8,3),'
    'vote_average       NUMERIC(5,3),'
    'vote_count         INT);'
)

In [473]:
df1.to_csv('df1.csv', header=False, index=False, sep='~')
cur.copy_from(f:=open('df1.csv', 'r', encoding='cp850'), 'tmdb_data', columns=['tmdb_id', 'imdb_id', 'title', 'original_language', 'release_date', 'popularity', 'vote_average', 'vote_count'], sep='~')

Note: Added overview and re-did this in the console.

In [466]:
df

,backdrop_path,id,original_title,overview,poster_path,media_type,adult,title,original_language,genre_ids,popularity,release_date,video,vote_average,vote_count,imdbid
0,/6vYffq6NoneNmNPp6IsobnnlrHS.jpg,315946,Passage de Venus,Photo sequence of the rare transit of Venus ov...,/XWPDZzK7N2WQcejI8W96IxZEeP.jpg,movie,False,Passage of Venus,fr,"(99,)",4.786,1874-12-09,False,6.210,105,3155794
0,NaN,766094,La Rosace Magique,Praxinoscope strip of a shifting rosette. Seri...,/5zeVMAp3R4QHGarSc4Bf3B1uBY2.jpg,movie,False,The Magic Rosette,xx,"(16,)",2.194,1878-05-07,False,5.800,19,14495706
0,NaN,751212,Le Singe Musicien,A pre-cinematograph colour animation of the mo...,/fx2gnyV2n9KdOCyGzQtVnpB9bIg.jpg,movie,False,The Musician Monkey,xx,"(16,)",2.560,1878-05-07,False,5.900,25,12592084
0,NaN,585297,L'Homme Machine,Animated stick drawings representing a man wal...,/vkxWQys2cqBU7JpWL4PWYG8Va9Z.jpg,movie,False,L'Homme Machine,xx,"(16,)",1.149,1885-01-01,False,4.629,31,8588366
0,/gCm6ggiw0YzIZkchHbDWrQbcOiP.jpg,159897,Man Walking Around a Corner,The last remaining production of Le Prince's L...,/3vIpxTUyCFSFagJsGaxFXcR8mOt.jpg,movie,False,Man Walking Around a Corner,xx,"(99,)",5.529,1887-08-18,False,4.900,80,2075247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,/wODqakS0jinTUECNS6n4VomQbew.jpg,967847,Ghostbusters: Frozen Empire,When the discovery of an ancient artifact unle...,/e1J2oNzSBdou01sUvriVuoYp0pJ.jpg,movie,False,Ghostbusters: Frozen Empire,en,"(14, 12, 35)",603.739,2024-03-20,False,6.671,873,21235248
0,/oe7mWkvYhK4PLRNAVSvonzyUXNy.jpg,359410,Road House,Ex-UFC fighter Dalton takes a job as a bouncer...,/bXi6IQiQDHD00JFio5ZSZOeRSBh.jpg,movie,False,Road House,en,"(28, 53)",483.627,2024-03-08,False,7.024,1810,3359350
0,/xRd1eJIDe7JHO5u4gtEYwGn5wtf.jpg,823464,Godzilla x Kong: The New Empire,"Following their explosive showdown, Godzilla a...",/z1p34vh7dEOnLDmyCrlUVLuoDzd.jpg,movie,False,Godzilla x Kong: The New Empire,en,"(878, 28, 12)",3853.790,2024-03-27,False,7.278,2211,14539740
0,/zVMdsmRUH2U1bZSYr8GLZkfs3mi.jpg,957304,Drive-Away Dolls,"Jamie, an uninhibited free spirit bemoaning ye...",/gavGnAMTXPkpoFgG0stwgIgKb64.jpg,movie,False,Drive-Away Dolls,en,"(35, 80)",81.501,2024-02-22,False,5.531,208,19356262


In [52]:
conn.commit()
cur.execute('SELECT * FROM tmdb_data;')
cur.fetchone()

(315946,
 3155794,
 'Passage of Venus',
 'fr',
 '1874-12-09',
 Decimal('4.786'),
 Decimal('6.210'),
 105)

### Genres

In [54]:
url = "https://api.themoviedb.org/3/genre/movie/list"
auth = 'Bearer ' + open('tmdbauth.txt').read() 
headers = {
    'accept' : 'application/json',
    'Authorization' : auth
}
response = requests.get(url, headers=headers)
genre_dict = {}
for genre in json.loads(response.text)['genres']:
    genre_dict[genre['id']] = genre['name']
genre_dict

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [81]:
df = df.set_index('imdbid')
ids, genres = [], []
for i in df.index:
    for j in df['genre_ids'][i]:
        try: ids.append(i)
        except: ids.append(np.nan)
        try: genres.append(genre_dict[j])
        except: genres.append(np.nan)
df2 = pd.DataFrame({'ids':ids, 'genres':genres}).dropna()

In [86]:
df2.to_csv('df2.csv', header=False, index=False, sep='~')
cur.execute(
    'CREATE TABLE genre('
    'imdb_id    INT,'
    'genre      VARCHAR(25)'
    ');'
)
cur.copy_from(f:=open('df2.csv', 'r'), 'genre', columns=['imdb_id', 'genre'], sep='~')

## Scraping Scripts
At this point, there is still a lot of uncertainty in terms of the data quality resulting from scraping scripts from IMSDb as I did in the previous notebook. In this section I will dig a little deeper to ensure only quality data gets uploaded to the database.

In [128]:
data = pd.read_csv('exploratory_data.csv')

Some of the issues to deal with before pushing to the Database are:
- Some of the scripts are blank between the pre and /pre tags
    - It looks like 25 scripts have this issue of the ~500 in the exploratory set
    - Checking on a few of them, it seems there is simply no script available for those movies, meaning that if the scraped HTML from IMSDb is blank in between the pre tags, it can be discarded. 
- Some scripts have multiple sets of pre tags
    - The innermost set usually contains the script
- It's possible that some scripts are available at different URLs- try scraping the script URL from the metadata page
- Release year should be matched up before scraping the script

### Blank Scripts (between the pre tags)

In [129]:
data['script'] = pd.Series()
for i in data.index:
    data['script'][i] = data['html'][i].split('<pre>')[-1].split('</pre>')[0]

C:\Users\Mark\AppData\Local\Temp\ipykernel_21516\2810471973.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['script'][i] = data['html'][i].split('<pre>')[-1].split('</pre>')[0]
C:\Users\Mark\AppData\Local\Temp\ipykernel_21516\2810471973.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['script'][i] = data['html'][i].split('<pre>')[-1].split('</pre>')[0]
C:\Users\Mark\AppData\Local\Temp\ipykernel_21516\2810471973.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [130]:
data['release_year'] = pd.Series()
data['release_month'] = pd.Series()
data['script_year'] = pd.Series()
data['script_month'] = pd.Series()

pattern = r'(.*)(\d\d\d\d)'
for i in data.index:
    try:
        match = re.search(pattern, data.loc[i].release_date)
        data['release_year'][i] = match[2]
        data['release_month'][i] = match[1]
    except TypeError: pass
    try:
        match = re.search(pattern, data.loc[i].script_date)
        data['script_year'][i] = match[2]
        data['script_month'][i] = match[1]
    except TypeError: pass

    


test = data[data['year'] == data['release_year'].fillna(0).astype(int)]#.fillna(0).astype(int)]
#data[data['script'] == '']

C:\Users\Mark\AppData\Local\Temp\ipykernel_21516\956866200.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['release_year'][i] = match[2]
C:\Users\Mark\AppData\Local\Temp\ipykernel_21516\956866200.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['release_month'][i] = match[1]
C:\Users\Mark\AppData\Local\Temp\ipykernel_21516\956866200.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['script_year'][i] = match[2

In [123]:
data[data['script'] != '']

,Unnamed: 0,id,imdbid,title,rating,year,html,release_date,script_date,script,release_year,release_month,script_year,script_month
0,202,1227,18033.0,It,3,1927,<html>\r\n<head>\r\n<!-- Google tag (gtag.js) ...,September 2017,March 2014,\r\n\r\n\r\n<b> ...,2017,September,2014,March
2,276,1328,22958.0,Grand Hotel,3,1932,<html>\r\n<head>\r\n<!-- Google tag (gtag.js) ...,April 1932,not listed,\r\n\r\n\r\n<b> ...,1932,April,NaN,NaN
3,292,6063,23427.0,Scarface,1,1932,<html>\r\n<head>\r\n<!-- Google tag (gtag.js) ...,not listed,not listed,"#00766\r\n\r\n\r\n\r\n\r\n ""Enjoy your...",NaN,NaN,NaN,NaN
4,416,174,32138.0,"Wizard of Oz, The",3,1939,<html>\r\n<head>\r\n<!-- Google tag (gtag.js) ...,not listed,March 1939,FADE IN -- Title:\r\n\r\nFor nearly forty year...,NaN,NaN,1939,March
5,481,1266,33467.0,Citizen Kane,1,1941,<html>\r\n<head>\r\n<!-- Google tag (gtag.js) ...,not listed,not listed,<html>\r\n<head>\r\n<script>\r\n<b><!--\r\n</b...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,9665,9265,4566758.0,Mulan,3,2020,<html>\r\n<head>\r\n<!-- Google tag (gtag.js) ...,not listed,December 1998,NaN,NaN,NaN,1998,December
537,9858,10052,1160419.0,Dune,3,2021,<html>\r\n<head>\r\n<!-- Google tag (gtag.js) ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,9971,10221,11245972.0,Scream,3,2022,<html>\r\n<head>\r\n<!-- Google tag (gtag.js) ...,not listed,July 1995,NaN,NaN,NaN,1995,July
539,10101,10684,10278918.0,Willow,3,2022,<html>\r\n<head>\r\n<!-- Google tag (gtag.js) ...,not listed,not listed,NaN,NaN,NaN,NaN,NaN


In [318]:
notFound = urlopen('https://imsdb.com/scripts/asd.html').read()
def scrape_imsdb(title, year, imdb_id, notFound=notFound):
    movie_year, script_date, script_url, script = -99, -99, -99, -99
    t = title.replace(' ', '%20')
    url = 'https://imsdb.com/Movie%20Scripts/' + t + '%20Script.html'
    try: 
        page = urlopen(url)
        html_bytes = page.read()
        html = html_bytes.decode('utf-8')
        
        movie_date_pattern = r'(<b>Movie Release Date</b> : )(.*)(\d\d\d\d)(<br>)'
        try: movie_year = re.search(movie_date_pattern, html)[3]
        except TypeError: movie_year = '9999'
        
        if (int(movie_year) == int(year)) or (int(movie_year) == 9999):
            script_date_pattern = r'(<b>Script Date</b> : )(.*)(<br>)'
            try: script_date = re.search(script_date_pattern, html)[2]
            except TypeError: pass
        
            script_url_pattern = r'(<a href=")(.*)(">Read &quot;)(.*)(&quot; Script</a>)'
            try: script_url = re.search(script_url_pattern, html)[2]
            except TypeError: pass
            
            script_url = 'https://imsdb.com' + script_url
            page = urlopen(script_url)
            html_bytes = page.read()
            if html_bytes == notFound:
                pass
            else:
                html = html_bytes.decode('utf-8')
                script = BeautifulSoup(html.split('<pre>')[-1].split('</pre>')[0]).get_text()
        
        
    except (HTTPError, UnicodeDecodeError, IndexError):
        pass
    finally:
        tup = (title, imdb_id, movie_year, script_date, script)
        return tup

In [349]:
'''
processes = []
with ThreadPoolExecutor(max_workers=32) as executor:
    for i in targets.index:
        processes.append(executor.submit(scrape_imsdb, targets['title'][i], targets['year'][i], targets['imdbid'][i]))


dict = {}
for task in processes:
    dict[task.result()[0]] =  (task.result()[1], task.result()[2], task.result()[3], task.result()[4])
'''

In [350]:
'''
df = pd.DataFrame.from_dict(dict).T
df.rename(columns={0:'imdb_id', 1:'release_year', 2:'script_date', 3:'script'}, inplace=True)
df = df[df['script'] != -99]
df.replace('9999', np.nan, inplace=True)
df.replace(-99, np.nan, inplace=True)
df.reset_index(inplace=True)
df.set_index('imdb_id', inplace=True)
df.rename(columns={'index':'title'}, inplace=True)
df.to_csv('imsdb_scripts.csv')
'''

In [463]:
df = pd.read_csv('imsdb_scripts.csv', index_col=0)
df = df.drop(columns=['title', 'release_year']).reset_index()
df.to_csv('scripts_upload.csv', header=False, index=False, na_rep='none')

Psycopg2 doesn't play nice with formatted text; the SQL to upload this table is in Table Creation.sql.

In [474]:
cur.close()
conn.close()

In [461]:
conn.rollback()

In [472]:
conn.commit()